# Model-based transfer experiments

This notebook contains experiments with model-based transfer techniques.

## Change working directory to project root

In [ ]:
import os
ROOT_DIRECTORIES = {'dogwood', 'tests'}
if set(os.listdir('.')).intersection(ROOT_DIRECTORIES) != ROOT_DIRECTORIES:
    os.chdir('../..')

## Exploration

In [ ]:
from mlops.dataset.versioned_dataset import VersionedDataset
from mlops.model.versioned_model import VersionedModel
from dogwood.pretraining.pretraining_pool import PretrainingPool

In [ ]:
pool = PretrainingPool()
print('Models:')
for model_path in pool.get_available_models():
    print(model_path)
print()
print('Datasets:')
for dataset_path in pool.get_available_datasets():
    print(dataset_path)

In [ ]:
dataset_imagenet = VersionedDataset(pool.get_dataset_path('imagenet-mini'))
model_vgg16 = VersionedModel(pool.get_model_path('VGG16'))
model_effnet = VersionedModel(pool.get_model_path('EfficientNetB7'))

In [ ]:
model_vgg16.model.summary()

In [ ]:
model_effnet.model.summary()